# Scraping China v2

New attempt to automatically collect data from a website bypassing its captcha for the new version of the website.

URL: http://43.248.49.97/

Direct URL for the form: http://43.248.49.97/queryDataForEN/queryDataByWhere

Requirement for using selenium for automating browser interaction:

- install browser driver:
https://sites.google.com/a/chromium.org/chromedriver/downloads

## User Requirements

Download import and export data for each month for oil commodities.

## Selenium strategy

1. Go to the website
1. Click 'Import and Export' radio button
1. Set Period
1. Check 'By month' box
1. On the first combo box, select 'Commodity'
1. Parse captcha
1. Set Captcha text to text box
1. Click 'Enquiry'


In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

#CHROME_PATH = '/usr/bin/google-chrome'
CHROMEDRIVER_PATH = 'C:\\Users\\ROSA_L\\Documents\\drivers\\chromedriver.exe'
#WINDOW_SIZE = "1920,1080"
CHINA_CUSTOMS_URL = "http://43.248.49.97/queryDataForEN/queryDataByWhere"


chrome_options = Options()  
#chrome_options.add_argument("--headless")  
#chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
#chrome_options.binary_location = CHROME_PATH

driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH,
                          chrome_options=chrome_options
                         )  

driver.get(CHINA_CUSTOMS_URL)
#driver.get_screenshot_as_file("capture.png")
#driver.close()

c:\users\rosa_l\pycharmprojects\scrapper\venv\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: use options instead of chrome_options
  app.launch_new_instance()


In [2]:
from selenium.webdriver.support.ui import Select
Select(driver.find_element_by_id("outerField1")).select_by_value("CODE_TS")
driver.find_element_by_id("outerValue1").send_keys("27111200")

In [3]:
driver.find_element_by_id("year").send_keys("2019")
driver.find_element_by_id("startMonth").send_keys("1")
driver.find_element_by_id("endMonth").send_keys("2")
driver.find_element_by_name("monthFlag").click()

In [4]:
driver.find_element_by_name("monthFlag").click()
driver.find_element_by_id("startMonth").send_keys("1")
driver.find_element_by_id("endMonth").send_keys("1")

In [5]:
# find radio buttons
radiosIE = driver.find_elements_by_name("iEType")

for r in radiosIE:
    if r.get_attribute("value") == "1":
        r.click()

In [55]:
from PIL import Image

def save_image(chrome, element_id, save_path):
  # in case the image isn't isn't in the view yet
  
  element = chrome.find_element_by_id(element_id)
    
  location = element.location_once_scrolled_into_view
  size = element.size

  # saves screenshot of entire page
  chrome.save_screenshot(save_path)
  # uses PIL library to open image in memory

  image = Image.open(save_path)
  left = location['x'] + 2
  top = location['y'] + 2
  right = location['x'] + size['width']
  bottom = location['y'] + size['height']
  image = image.crop((left, top, right, bottom))  # defines crop points
  image.save(save_path, 'png')  # saves new cropped image

save_image(chrome=driver, element_id="img", save_path="C:\\Users\\ROSA_L\\Documents\\scraping_china\\test_captcha2.png")


In [6]:
driver.find_element_by_id("doSearch").click()

In [89]:
import re
from selenium.common.exceptions import UnexpectedAlertPresentException

regex_num = re.compile('\d+')

try:
    results = []
    # extract data from table
    rows = driver.find_elements_by_xpath(".//div[@class='datagrid-view2']//div[@class='datagrid-body']//tr")


    for row in rows:
        result = {}
        JSON_con1 = row.find_element_by_xpath("./td[@field='JSON_con1']").text
        result['commodity_code'] = int(regex_num.match(JSON_con1).group(0))

        JSON_qty = row.find_element_by_xpath("./td[@field='JSON_qty']").text
        result['quantity'] = int(JSON_qty)
        results.append(result)
except UnexpectedAlertPresentException:
    print("WARNING: Alert when reading results. Retry!")
    alert = driver.switch_to.alert
    alert.accept()
results


[]

In [90]:
driver.close()

In [53]:
# clear
driver.find_element_by_id("TextBox_fenzu1").clear()
driver.find_element_by_id("verifytext").clear()

In [7]:
flag = driver.find_element_by_id("successFlag")
flag is not None and flag.is_displayed()

False

In [ ]:
import pandas as pd
import time
from selenium.webdriver.support.ui import Select

# find resulting html table
driver.switch_to.default_content()
iframe = driver.find_element_by_xpath('//iframe[@src="/queryDataForEN/queryDataByWhere"]')
driver.switch_to.frame(iframe);
table = driver.find_element_by_id("table").find_element_by_xpath('..').get_attribute('innerHTML')

# change page size to max
Select(driver.find_element_by_id("pageSize")).select_by_value("200")
time.sleep(60)

cond = True

while cond:
    dfs = pd.read_html(table, header=0)
    print(dfs[0])
    next_button = driver.find_element_by_link_text("Next page")
    cond = not next_button.get_attribute("class").contains("disabled")
    if cond:
        next_button.click()
        time.sleep(60)

In [62]:
len(dfs[0].index)

52

In [66]:
next_button = driver.find_element_by_link_text("Next page")
next_button.is_enabled()

True

In [94]:
from selenium.common.exceptions import *

try:
  messagebox = driver.find_element_by_id("layui-layer1")
  if messagebox.is_displayed():
    button = driver.find_element_by_link_text("Confirm")
    print("message box visible")
    button.click()
except NoSuchElementException:
    print('No such element.')
    pass

No such element.


In [11]:
# find 'Another one' button
button = driver.find_element_by_link_text("Another one")
button.click()


In [79]:
import re
regex_num = re.compile('\d+')
text = '27090000石油原油及从沥青矿物提取的原油'
int(regex_num.match(text).group(0))

27090000

In [104]:
import pandas as pd

excelpath = "C:\\Users\\ROSA_L\\Documents\\scraping_china\\codes_to_extract.xlsx"
wks = "CODES_TO_EXTRACT"

df = pd.read_excel(excelpath, sheet_name=wks)
df = df[["Product_code"]]
liste = df["Product_code"].tolist
liste

<bound method IndexOpsMixin.tolist of 0     27090000
1     27111200
2     27111310
3     27111390
4     27101220
5     27101210
6     27101299
7     27101911
8     27101912
9     27101919
10    27101923
11    27101922
12    27101929
13    27101991
14    27101992
15    27101993
16    27101994
17    27122000
18    27131110
19    27131190
20    27131210
21    27131290
22    27132000
23    27101999
24    27075000
25    27079990
26    29091990
Name: Product_code, dtype: int64>

In [118]:
total = 3
nb_it = int(total / 5) + int(total % 5 > 0)

nb_it


1

In [110]:
l = [7090000,27111200,27111310,27111390,27101220]
','.join([str(x) for x in l])

'7090000,27111200,27111310,27111390,27101220'

In [169]:
import time 
str_ts = time.strftime("%Y%m%d%H%M", time.gmtime())
str_ts

'201902211323'

In [115]:
import os
os.getcwd()

'C:\\Users\\ROSA_L\\Documents'

In [117]:
str = 'blabla'
str[:-1]

'blabl'

In [141]:
import re
pat=re.compile(r"(19|20)\d\d-(0[1-9]|1[012])$")
if not pat.match("2018-09"):
    print("ko")
else:
    print("ok")


ok


In [139]:
import re
import argparse

def my_regex_type(s, pat=re.compile(r"((19|20)\d\d-0[1-9]|1[012])$")):
    if not pat.match(s):
        raise argparse.ArgumentTypeError
    return s

parser = argparse.ArgumentParser()
parser.add_argument('start-period', type=my_regex_type)

args = parser.parse_args(["2018"])

usage: ipykernel_launcher.py [-h] start-period
ipykernel_launcher.py: error: argument start-period: 


SystemExit: 2

C:\Users\ROSA_L\AppData\Local\Continuum\Anaconda3_x86\lib\site-packages\IPython\core\interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [162]:
def month_year_iter( start_period, end_period ):
    YEAR  = 0
    MONTH = 1
    #split_yyyy_mm = lambda string: list(map(int, string.split('-')))
    def split_yyyy_mm(string): return [int(x) for x in string.split('-')]

    start = split_yyyy_mm(start_period)
    end   = split_yyyy_mm(end_period)
    
    ym_start= 12*start[YEAR] + start[MONTH] - 1
    ym_end= 12*end[YEAR] + end[MONTH] 

    for ym in range( ym_start, ym_end):
        y, m = divmod( ym, 12 )
        yield f"{y}-{m+1:02d}"

for period in month_year_iter("2018-01", "2019-02" ):
    print(f'Period: {period}')

Period: 2018-01
Period: 2018-02
Period: 2018-03
Period: 2018-04
Period: 2018-05
Period: 2018-06
Period: 2018-07
Period: 2018-08
Period: 2018-09
Period: 2018-10
Period: 2018-11
Period: 2018-12
Period: 2019-01
Period: 2019-02


In [164]:
"file.png"[:-4]

'file'

In [165]:
import struct;print( 8 * struct.calcsize("P"))

32


In [166]:
import platform
platform.system()

'Windows'

In [184]:
def save_results(start_period, end_period):

    ts = time.strftime("%Y%m%d%H%M", time.gmtime())
    filename = f"blabla/{ts}_ooutput_{start_period}" +\
    (f"_to_{end_period}" if end_period else "") +\
    ".png"
    print(filename)

save_results("2019-02", "2019-03")
print('test')
save_results("2019-02", None)

blabla/201902211339_ooutput_2019-02_to_2019-03.png
test
blabla/201902211339_ooutput_2019-02.png


In [198]:
import random

for i in range(0,10):
    print(random.randint(1,5))

3
3
5
1
3
5
2
1
4
5


In [205]:
import os
p = "C:\\Users\\ROSA_L\\PycharmProjects\\scrapper\\refinery\\china\\images"

filename = p + "\\0R8Z.png"
guess = "6T40"

def rename_img(filename, guess):
cnt = 1
newname = f"{p}\\{guess}.png"
while True:
    try:
        print("trying to rename...")
        os.rename(filename, newname)
        break
    except FileExistsError:
        print(f"retryng nb {cnt}...")
        newname = f"{p}\\{guess}.{cnt}.png"
        cnt += 1

trying to rename...
retryng nb 1...
trying to rename...


In [ ]:
from PIL import Image
import cv2
import numpy as np

img = Image.open('C:\\Users\\ROSA_L\\Pictures\\SCCv.jpg')
img = np.asarray(img)
#print("CAPTCHA text is: {}".format(captcha_text))
#newname = CAPTCHA_IMAGE_FOLDER + "/" + captcha_text + ".gif"
#os.rename(filename,newname)
# Show the annotated image
cv2.imshow("Output", img)
cv2.waitKey()

In [2]:
'nn_solver'.split('_')

['nn', 'solver']

In [228]:
# importing required packages
import pandas as pd
import pandas_profiling
import numpy as np

# importing the data
df = pd.read_csv('C:\\Users\\ROSA_L\\PycharmProjects\\scrapper\\filestore\\cn_customs_crudeoil_imp_2017.csv', thousands=',')

# descriptive statistics
#df.describe()
#pandas_profiling.ProfileReport(df)

In [229]:
df2 = pd.read_csv('C:\\Users\\ROSA_L\\PycharmProjects\\scrapper\\filestore\\cn_customs_crudeoil_imp_2018.csv', thousands=',')
df3 = pd.read_csv('C:\\Users\\ROSA_L\\PycharmProjects\\scrapper\\filestore\\cn_customs_crudeoil_imp_201901.csv', thousands=',')
df4 = pd.read_csv('C:\\Users\\ROSA_L\\PycharmProjects\\scrapper\\filestore\\cn_customs_crudeoil_imp_201902.csv', thousands=',')
df5 = pd.read_csv('C:\\Users\\ROSA_L\\PycharmProjects\\scrapper\\filestore\\cn_customs_crudeoil_imp_201903.csv', thousands=',')

In [230]:
df = pd.concat([df, df2, df3, df4, df5])

c:\users\rosa_l\pycharmprojects\scrapper\venv\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [233]:
df['Renminbi Yuan']

0       225969098
1       223023436
2       211470645
3       213317112
4       474474873
5       124029003
6       215016712
7            1210
8            1212
9       463325505
10      110821253
11      271888511
12      199077092
13      153864166
14      779867818
15      868805468
16      387198837
17      281314791
18      639283460
19     4686887552
20     6794841250
21     5526258647
22     7562055486
23     7356332054
24     5302381063
25     7908752775
26     7161756274
27     6866230103
28     7592740975
29     7808556083
         ...     
4       174672489
5      9477890872
6       228320160
7     23884716836
8       355892990
9      6000871735
10      349566157
11      595262854
12      757900201
13      508172627
14     9780186319
15      844538600
16     2554433146
17     1004029627
18     2116130275
19      848412945
20     2241193795
21      213016516
22      404033803
23      971212576
24     2262088329
25      135616347
26    17073863308
27      460408564
28     933

In [14]:
import pandas as pd
df_map = pd.read_excel('C:\\Users\ROSA_L\\PycharmProjects\\scrapper\\scraper\\jobs\\cn_customs\\cn_customs_mappings.xlsx', sheet_name='PRODUCTS')

In [17]:
df_map

,Product_code,Product_desc_ch,Product_desc_en,external_db_code
0,27090000,石油原油及从沥青矿物提取的原油,Petroleum oils&oils obtained from bituminous m...,CRUDEOIL
1,27111200,液化丙烷,"Propane, liquefied",LPG
2,27111310,打火机用液化丁烷，容器容积＞300cucm,Liquefd butane in container>300 cm3 for fillng...,LPG
3,27111390,其他液化丁烷,"Liquefied butanes, nes",LPG
4,27101220,石脑油，不含有生物柴油,"Naphtha, without biodiesel",NAPHTHA
5,27101210,车用汽油和航空汽油，不含有生物柴油,"Motor/aviation gasoline, without biodiesel",MOTORGAS
6,27101299,未列名轻油及其制品，不含有生物柴油,"Other light oils and preparations, without bio...",MOTORGAS
7,27101911,航空煤油，不含有生物柴油,"Aviation kerosene, without biodiesel",JETKERO
8,27101912,灯用煤油，不含生物柴油,"Lamp-kerosene, without biodiesel",OTHKERO
9,27101919,其他煤油馏分，不含有生物柴油,"Other kerosene distillages, without biodiesel",OTHKERO


In [225]:
df_m = pd.merge(df, df_map, left_on='Trading partner code', right_on='country_code',  how='left', indicator='result_type')
len(df_m)

TypeError: merge() got an unexpected keyword argument 'thousands'

In [44]:
len(df)

937

In [224]:
df_m

,Commodity,Commodity code,Date of data,Quantity,Renminbi Yuan,Supplimentary Quantity,Supplimentary Unit,Trading partner,Trading partner code,Unit,Unnamed: 10,Unnamed: 9,country_code,country_name_cn,country_name_en,code,result_type
0,Petroleum oils&oils obtained from bituminous m...,27090000,201709.0,84696000,"225,969,098",0,NaN,Brunei,105,NaN,NaN,NaN,105,文莱,Brunei,BRUNEI,both
1,Petroleum oils&oils obtained from bituminous m...,27090000,201703.0,74416591,"223,023,436",0,NaN,Brunei,105,NaN,NaN,NaN,105,文莱,Brunei,BRUNEI,both
2,Petroleum oils&oils obtained from bituminous m...,27090000,201706.0,78932000,"211,470,645",0,NaN,Brunei,105,NaN,NaN,NaN,105,文莱,Brunei,BRUNEI,both
3,Petroleum oils&oils obtained from bituminous m...,27090000,201704.0,79557316,"213,317,112",0,NaN,Brunei,105,NaN,NaN,NaN,105,文莱,Brunei,BRUNEI,both
4,Petroleum oils&oils obtained from bituminous m...,27090000,201701.0,157883252,"474,474,873",0,NaN,Brunei,105,NaN,NaN,NaN,105,文莱,Brunei,BRUNEI,both
5,Petroleum oils&oils obtained from bituminous m...,27090000,201702.0,41455884,"124,029,003",0,NaN,Brunei,105,NaN,NaN,NaN,105,文莱,Brunei,BRUNEI,both
6,Petroleum oils&oils obtained from bituminous m...,27090000,201708.0,76687000,"215,016,712",0,NaN,Brunei,105,NaN,NaN,NaN,105,文莱,Brunei,BRUNEI,both
7,Petroleum oils&oils obtained from bituminous m...,27090000,201709.0,6,"1,210",0,NaN,India,111,NaN,NaN,NaN,111,印度,India,INDIA,both
8,Petroleum oils&oils obtained from bituminous m...,27090000,201712.0,9,"1,212",0,NaN,Indonesia,112,NaN,NaN,NaN,112,印度尼西亚,Indonesia,INDONESIA,both
9,Petroleum oils&oils obtained from bituminous m...,27090000,201703.0,159324616,"463,325,505",0,NaN,Indonesia,112,NaN,NaN,NaN,112,印度尼西亚,Indonesia,INDONESIA,both


In [50]:
df_rej = df_m[df_m['result_type'] == 'left']

In [51]:
df_ok = df_m[df_m['result_type'] == 'both']

In [62]:
df.dropna(axis='columns', how='all', inplace=True)

In [64]:
df['Supplimentary Unit'].drop_duplicates()

0    NaN
0      -
Name: Supplimentary Unit, dtype: object

In [63]:
import logging
logging.basicConfig(level = logging.DEBUG)

from scraper.core.job import BATCH_SIZE

import scraper.jobs.cn_customs.oil_products_job as job


logger = logging.getLogger(__name__)

logger.debug('test DEBUG')
logger.info('test INFO')

cn = job.OilProductsJob(full_load=True)
cn.run()

DEBUG:__main__:test DEBUG
INFO:__main__:test INFO
DEBUG:scraper.jobs.cn_customs.oil_products_job:Getting sources...
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): vipenta:8000
DEBUG:urllib3.connectionpool:http://vipenta:8000 "GET /dimension/source HTTP/1.1" 200 584495
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): vipenta:8000
DEBUG:urllib3.connectionpool:http://vipenta:8000 "GET /dimension/source HTTP/1.1" 200 584495
INFO:scraper.core.utils:download_and_get_checksum: 65.04654884338379 ms
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): vipenta:8000
DEBUG:urllib3.connectionpool:http://vipenta:8000 "GET /dimension/source?code=cn_customs_oilprod_exp_2017 HTTP/1.1" 200 677
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): vipenta:8000
DEBUG:urllib3.connectionpool:http://vipenta:8000 "GET /dimension/source?code=cn_customs_oilprod_exp_2018 HTTP/1.1" 200 677
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): vipenta:8000


In [22]:
cn.dynamic_dim['source']

[{'code': 'cn_customs_oilprod_imp_201906',
  'url': 'http://43.248.49.97/indexEn',
  'path': 'cn_customs_oilprod_imp_201906.csv',
  'long_name': 'CHINA CN_CUSTOMS Oil products imports - 201906'},
 {'code': 'cn_customs_oilprod_exp_201906',
  'url': 'http://43.248.49.97/indexEn',
  'path': 'cn_customs_oilprod_exp_201906.csv',
  'long_name': 'CHINA CN_CUSTOMS Oil products exports - 201906'}]

In [29]:
import pandas as pd
df_map = pd.read_excel('C:\\Users\ROSA_L\\PycharmProjects\\scrapper\\scraper\\jobs\\cn_customs\\cn_customs_mappings.xlsx', sheet_name='PRODUCTS')

In [30]:
df_map

,Product_code,Product_desc_ch,Product_desc_en,external_db_code
0,27090000,石油原油及从沥青矿物提取的原油,Petroleum oils&oils obtained from bituminous m...,CRUDEOIL
1,27111200,液化丙烷,"Propane, liquefied",LPG
2,27111310,打火机用液化丁烷，容器容积＞300cucm,Liquefd butane in container>300 cm3 for fillng...,LPG
3,27111390,其他液化丁烷,"Liquefied butanes, nes",LPG
4,27101220,石脑油，不含有生物柴油,"Naphtha, without biodiesel",NAPHTHA
5,27101210,车用汽油和航空汽油，不含有生物柴油,"Motor/aviation gasoline, without biodiesel",MOTORGAS
6,27101299,未列名轻油及其制品，不含有生物柴油,"Other light oils and preparations, without bio...",MOTORGAS
7,27101911,航空煤油，不含有生物柴油,"Aviation kerosene, without biodiesel",JETKERO
8,27101912,灯用煤油，不含生物柴油,"Lamp-kerosene, without biodiesel",OTHKERO
9,27101919,其他煤油馏分，不含有生物柴油,"Other kerosene distillages, without biodiesel",OTHKERO


In [36]:
df = pd.read_csv('C:\\Users\\ROSA_L\\PycharmProjects\\scrapper\\filestore\\cn_customs_oilprod_imp_201906.csv', encoding='windows-1252', thousands=',', na_values='-')
df['source']='cn_customs_oilprod_imp_201906'
df['flow']='IMPORTS'
df

,Date of data,Commodity code,Commodity,Quantity,Unit,Supplimentary Quantity,Supplimentary Unit,Renminbi Yuan,Unnamed: 8,source,flow
0,201906,27075000,Other aromatic hydrocarbon mixtures,52369497,Kilogram,0,NaN,210757400,NaN,cn_customs_oilprod_imp_201906,IMPORTS
1,201906,27079990,"Prdcts frm high tmprtr coal tar; prdcts, aroma...",292172224,Kilogram,0,NaN,1154809846,NaN,cn_customs_oilprod_imp_201906,IMPORTS
2,201906,27090000,Petroleum oils&oils obtained from bituminous m...,39579392613,Kilogram,0,NaN,138336235253,NaN,cn_customs_oilprod_imp_201906,IMPORTS
3,201906,27101210,"Motor/aviation gasoline, without biodiesel",37718412,Kilogram,52353157,Litre,146140963,NaN,cn_customs_oilprod_imp_201906,IMPORTS
4,201906,27101220,"Naphtha, without biodiesel",255950510,Kilogram,354491456,Litre,973502714,NaN,cn_customs_oilprod_imp_201906,IMPORTS
5,201906,27101299,"Other light oils and preparations, without bio...",16161686,Kilogram,0,NaN,79593221,NaN,cn_customs_oilprod_imp_201906,IMPORTS
6,201906,27101911,"Aviation kerosene, without biodiesel",283042498,Kilogram,352670955,Litre,1272558050,NaN,cn_customs_oilprod_imp_201906,IMPORTS
7,201906,27101919,"Other kerosene distillages, without biodiesel",4731779,Kilogram,5360662,Litre,33124365,NaN,cn_customs_oilprod_imp_201906,IMPORTS
8,201906,27101922,"Fuel oils No. 5-No. 7, without biodiesel",974076173,Kilogram,988687316,Litre,3006068471,NaN,cn_customs_oilprod_imp_201906,IMPORTS
9,201906,27101923,Diesel oils,136542766,Kilogram,160574292,Litre,521186726,NaN,cn_customs_oilprod_imp_201906,IMPORTS


In [37]:
df.dropna(axis='columns', how='all', inplace=True)

In [39]:
df

,Date of data,Commodity code,Commodity,Quantity,Unit,Supplimentary Quantity,Supplimentary Unit,Renminbi Yuan,source,flow
0,201906,27075000,Other aromatic hydrocarbon mixtures,52369497,Kilogram,0,NaN,210757400,cn_customs_oilprod_imp_201906,IMPORTS
1,201906,27079990,"Prdcts frm high tmprtr coal tar; prdcts, aroma...",292172224,Kilogram,0,NaN,1154809846,cn_customs_oilprod_imp_201906,IMPORTS
2,201906,27090000,Petroleum oils&oils obtained from bituminous m...,39579392613,Kilogram,0,NaN,138336235253,cn_customs_oilprod_imp_201906,IMPORTS
3,201906,27101210,"Motor/aviation gasoline, without biodiesel",37718412,Kilogram,52353157,Litre,146140963,cn_customs_oilprod_imp_201906,IMPORTS
4,201906,27101220,"Naphtha, without biodiesel",255950510,Kilogram,354491456,Litre,973502714,cn_customs_oilprod_imp_201906,IMPORTS
5,201906,27101299,"Other light oils and preparations, without bio...",16161686,Kilogram,0,NaN,79593221,cn_customs_oilprod_imp_201906,IMPORTS
6,201906,27101911,"Aviation kerosene, without biodiesel",283042498,Kilogram,352670955,Litre,1272558050,cn_customs_oilprod_imp_201906,IMPORTS
7,201906,27101919,"Other kerosene distillages, without biodiesel",4731779,Kilogram,5360662,Litre,33124365,cn_customs_oilprod_imp_201906,IMPORTS
8,201906,27101922,"Fuel oils No. 5-No. 7, without biodiesel",974076173,Kilogram,988687316,Litre,3006068471,cn_customs_oilprod_imp_201906,IMPORTS
9,201906,27101923,Diesel oils,136542766,Kilogram,160574292,Litre,521186726,cn_customs_oilprod_imp_201906,IMPORTS


In [43]:
df_map = df_map[~df_map['external_db_code'].isnull()]
len(df_map)

27

In [44]:
 df_m = pd.merge(df, df_map, left_on='Commodity code', right_on='Product_code', how='left',
                        indicator='result_type')
      

In [45]:
df_m

,Date of data,Commodity code,Commodity,Quantity,Unit,Supplimentary Quantity,Supplimentary Unit,Renminbi Yuan,source,flow,Product_code,Product_desc_ch,Product_desc_en,external_db_code,result_type
0,201906,27075000,Other aromatic hydrocarbon mixtures,52369497,Kilogram,0,NaN,210757400,cn_customs_oilprod_imp_201906,IMPORTS,27075000,其他芳烃混合物，根据ISO 3405方法温度在250℃时的馏出量以体积计（包括损耗）≥65％,Other aromatic hydrocarbon mixtures,MOTORGAS,both
1,201906,27079990,"Prdcts frm high tmprtr coal tar; prdcts, aroma...",292172224,Kilogram,0,NaN,1154809846,cn_customs_oilprod_imp_201906,IMPORTS,27079990,其他蒸馏高温煤焦油所得油类等产品及类似品,"Prdcts frm high tmprtr coal tar; prdcts, aroma...",DIESEL,both
2,201906,27090000,Petroleum oils&oils obtained from bituminous m...,39579392613,Kilogram,0,NaN,138336235253,cn_customs_oilprod_imp_201906,IMPORTS,27090000,石油原油及从沥青矿物提取的原油,Petroleum oils&oils obtained from bituminous m...,CRUDEOIL,both
3,201906,27101210,"Motor/aviation gasoline, without biodiesel",37718412,Kilogram,52353157,Litre,146140963,cn_customs_oilprod_imp_201906,IMPORTS,27101210,车用汽油和航空汽油，不含有生物柴油,"Motor/aviation gasoline, without biodiesel",MOTORGAS,both
4,201906,27101220,"Naphtha, without biodiesel",255950510,Kilogram,354491456,Litre,973502714,cn_customs_oilprod_imp_201906,IMPORTS,27101220,石脑油，不含有生物柴油,"Naphtha, without biodiesel",NAPHTHA,both
5,201906,27101299,"Other light oils and preparations, without bio...",16161686,Kilogram,0,NaN,79593221,cn_customs_oilprod_imp_201906,IMPORTS,27101299,未列名轻油及其制品，不含有生物柴油,"Other light oils and preparations, without bio...",MOTORGAS,both
6,201906,27101911,"Aviation kerosene, without biodiesel",283042498,Kilogram,352670955,Litre,1272558050,cn_customs_oilprod_imp_201906,IMPORTS,27101911,航空煤油，不含有生物柴油,"Aviation kerosene, without biodiesel",JETKERO,both
7,201906,27101919,"Other kerosene distillages, without biodiesel",4731779,Kilogram,5360662,Litre,33124365,cn_customs_oilprod_imp_201906,IMPORTS,27101919,其他煤油馏分，不含有生物柴油,"Other kerosene distillages, without biodiesel",OTHKERO,both
8,201906,27101922,"Fuel oils No. 5-No. 7, without biodiesel",974076173,Kilogram,988687316,Litre,3006068471,cn_customs_oilprod_imp_201906,IMPORTS,27101922,5~7号燃料油，不含有生物柴油,"Fuel oils No. 5-No. 7, without biodiesel",RESFUEL,both
9,201906,27101923,Diesel oils,136542766,Kilogram,160574292,Litre,521186726,cn_customs_oilprod_imp_201906,IMPORTS,27101923,"柴油,不含生物柴油",Diesel oils,DIESEL,both


In [47]:
df_rej = df_m[(df_m['result_type'] == 'left_only') | (df_m['external_db_code'].isnull())]
df_rej

,Date of data,Commodity code,Commodity,Quantity,Unit,Supplimentary Quantity,Supplimentary Unit,Renminbi Yuan,source,flow,Product_code,Product_desc_ch,Product_desc_en,external_db_code,result_type


In [49]:
df_rej = df_m[(df_m['result_type'] == 'left_only') | (df_m['external_db_code'].isnull())]
df_rej

,Date of data,Commodity code,Commodity,Quantity,Unit,Supplimentary Quantity,Supplimentary Unit,Renminbi Yuan,source,flow,Product_code,Product_desc_ch,Product_desc_en,external_db_code,result_type


In [51]:
from calendar import monthrange
def convert_m3_to_kbd(value: float, year: int, month: int):
    d_in_month = monthrange(year, month)[1]
    value = round(value * 6.289811 / 1000 / d_in_month, 3)
    return value


df_m['period'] = df_m['Date of data'].apply(lambda x: pd.to_datetime(x, format='%Y%m')) \
                                             .dt.strftime("%b%Y").str.upper()
df_m['detail'] = 'cn_customs_' + df_m['external_db_code'] + '_' + df_m['Product_code'].astype('str')
df_m['KT'] = df_m['Quantity'] / 10 ** 6
df_m['KBD'] = df_m.apply(lambda x:
                                 convert_m3_to_kbd(x['Supplimentary Quantity']/1000,
                                                   x['Date of data'] // 100,
                                                   x['Date of data'] % 100)
                                 , axis=1)
df_m


,Date of data,Commodity code,Commodity,Quantity,Unit,Supplimentary Quantity,Supplimentary Unit,Renminbi Yuan,source,flow,Product_code,Product_desc_ch,Product_desc_en,external_db_code,result_type,period,detail,KT,KBD
0,201906,27075000,Other aromatic hydrocarbon mixtures,52369497,Kilogram,0,NaN,210757400,cn_customs_oilprod_imp_201906,IMPORTS,27075000,其他芳烃混合物，根据ISO 3405方法温度在250℃时的馏出量以体积计（包括损耗）≥65％,Other aromatic hydrocarbon mixtures,MOTORGAS,both,JUN2019,cn_customs_MOTORGAS_27075000,52.369497,0.000
1,201906,27079990,"Prdcts frm high tmprtr coal tar; prdcts, aroma...",292172224,Kilogram,0,NaN,1154809846,cn_customs_oilprod_imp_201906,IMPORTS,27079990,其他蒸馏高温煤焦油所得油类等产品及类似品,"Prdcts frm high tmprtr coal tar; prdcts, aroma...",DIESEL,both,JUN2019,cn_customs_DIESEL_27079990,292.172224,0.000
2,201906,27090000,Petroleum oils&oils obtained from bituminous m...,39579392613,Kilogram,0,NaN,138336235253,cn_customs_oilprod_imp_201906,IMPORTS,27090000,石油原油及从沥青矿物提取的原油,Petroleum oils&oils obtained from bituminous m...,CRUDEOIL,both,JUN2019,cn_customs_CRUDEOIL_27090000,39579.392613,0.000
3,201906,27101210,"Motor/aviation gasoline, without biodiesel",37718412,Kilogram,52353157,Litre,146140963,cn_customs_oilprod_imp_201906,IMPORTS,27101210,车用汽油和航空汽油，不含有生物柴油,"Motor/aviation gasoline, without biodiesel",MOTORGAS,both,JUN2019,cn_customs_MOTORGAS_27101210,37.718412,10.976
4,201906,27101220,"Naphtha, without biodiesel",255950510,Kilogram,354491456,Litre,973502714,cn_customs_oilprod_imp_201906,IMPORTS,27101220,石脑油，不含有生物柴油,"Naphtha, without biodiesel",NAPHTHA,both,JUN2019,cn_customs_NAPHTHA_27101220,255.950510,74.323
5,201906,27101299,"Other light oils and preparations, without bio...",16161686,Kilogram,0,NaN,79593221,cn_customs_oilprod_imp_201906,IMPORTS,27101299,未列名轻油及其制品，不含有生物柴油,"Other light oils and preparations, without bio...",MOTORGAS,both,JUN2019,cn_customs_MOTORGAS_27101299,16.161686,0.000
6,201906,27101911,"Aviation kerosene, without biodiesel",283042498,Kilogram,352670955,Litre,1272558050,cn_customs_oilprod_imp_201906,IMPORTS,27101911,航空煤油，不含有生物柴油,"Aviation kerosene, without biodiesel",JETKERO,both,JUN2019,cn_customs_JETKERO_27101911,283.042498,73.941
7,201906,27101919,"Other kerosene distillages, without biodiesel",4731779,Kilogram,5360662,Litre,33124365,cn_customs_oilprod_imp_201906,IMPORTS,27101919,其他煤油馏分，不含有生物柴油,"Other kerosene distillages, without biodiesel",OTHKERO,both,JUN2019,cn_customs_OTHKERO_27101919,4.731779,1.124
8,201906,27101922,"Fuel oils No. 5-No. 7, without biodiesel",974076173,Kilogram,988687316,Litre,3006068471,cn_customs_oilprod_imp_201906,IMPORTS,27101922,5~7号燃料油，不含有生物柴油,"Fuel oils No. 5-No. 7, without biodiesel",RESFUEL,both,JUN2019,cn_customs_RESFUEL_27101922,974.076173,207.289
9,201906,27101923,Diesel oils,136542766,Kilogram,160574292,Litre,521186726,cn_customs_oilprod_imp_201906,IMPORTS,27101923,"柴油,不含生物柴油",Diesel oils,DIESEL,both,JUN2019,cn_customs_DIESEL_27101923,136.542766,33.666


In [52]:
        df_m = df_m[['source', 'period', 'flow', 'external_db_code', 'detail',
                     'Supplimentary Unit', 'Supplimentary Quantity',
                     'KT', 'KBD', 'Renminbi Yuan']]\
            .rename(columns={'external_db_code': 'product',
                             'Renminbi Yuan': 'NC'})

In [53]:
df_m

,source,period,flow,product,detail,Supplimentary Unit,Supplimentary Quantity,KT,KBD,NC
0,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,MOTORGAS,cn_customs_MOTORGAS_27075000,NaN,0,52.369497,0.000,210757400
1,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,DIESEL,cn_customs_DIESEL_27079990,NaN,0,292.172224,0.000,1154809846
2,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,CRUDEOIL,cn_customs_CRUDEOIL_27090000,NaN,0,39579.392613,0.000,138336235253
3,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,MOTORGAS,cn_customs_MOTORGAS_27101210,Litre,52353157,37.718412,10.976,146140963
4,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,NAPHTHA,cn_customs_NAPHTHA_27101220,Litre,354491456,255.950510,74.323,973502714
5,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,MOTORGAS,cn_customs_MOTORGAS_27101299,NaN,0,16.161686,0.000,79593221
6,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,JETKERO,cn_customs_JETKERO_27101911,Litre,352670955,283.042498,73.941,1272558050
7,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,OTHKERO,cn_customs_OTHKERO_27101919,Litre,5360662,4.731779,1.124,33124365
8,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,RESFUEL,cn_customs_RESFUEL_27101922,Litre,988687316,974.076173,207.289,3006068471
9,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,DIESEL,cn_customs_DIESEL_27101923,Litre,160574292,136.542766,33.666,521186726


In [55]:
df_m = df_m.melt(id_vars=['source', 'period', 'flow', 'product', 'detail',
                                  'Supplimentary Unit', 'Supplimentary Quantity'],
                         value_vars=['KT', 'KBD', 'NC'],
                         var_name="unit",
                         value_name="value")
df_m

,source,period,flow,product,detail,Supplimentary Unit,Supplimentary Quantity,unit,value
0,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,MOTORGAS,cn_customs_MOTORGAS_27075000,NaN,0,KT,5.236950e+01
1,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,DIESEL,cn_customs_DIESEL_27079990,NaN,0,KT,2.921722e+02
2,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,CRUDEOIL,cn_customs_CRUDEOIL_27090000,NaN,0,KT,3.957939e+04
3,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,MOTORGAS,cn_customs_MOTORGAS_27101210,Litre,52353157,KT,3.771841e+01
4,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,NAPHTHA,cn_customs_NAPHTHA_27101220,Litre,354491456,KT,2.559505e+02
5,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,MOTORGAS,cn_customs_MOTORGAS_27101299,NaN,0,KT,1.616169e+01
6,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,JETKERO,cn_customs_JETKERO_27101911,Litre,352670955,KT,2.830425e+02
7,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,OTHKERO,cn_customs_OTHKERO_27101919,Litre,5360662,KT,4.731779e+00
8,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,RESFUEL,cn_customs_RESFUEL_27101922,Litre,988687316,KT,9.740762e+02
9,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,DIESEL,cn_customs_DIESEL_27101923,Litre,160574292,KT,1.365428e+02


In [56]:
df_m.sort_values(by=["product","detail"])

,source,period,flow,product,detail,Supplimentary Unit,Supplimentary Quantity,unit,value
24,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,ADDITIVES,cn_customs_ADDITIVES_29091990,NaN,0,KT,1.404600e-01
49,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,ADDITIVES,cn_customs_ADDITIVES_29091990,NaN,0,KBD,0.000000e+00
74,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,ADDITIVES,cn_customs_ADDITIVES_29091990,NaN,0,NC,1.592734e+07
15,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,BITUMEN,cn_customs_BITUMEN_27101999,Litre,4851400,KT,4.779710e+00
40,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,BITUMEN,cn_customs_BITUMEN_27101999,Litre,4851400,KBD,1.017000e+00
65,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,BITUMEN,cn_customs_BITUMEN_27101999,Litre,4851400,NC,5.387863e+07
23,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,BITUMEN,cn_customs_BITUMEN_27132000,NaN,0,KT,3.575259e+02
48,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,BITUMEN,cn_customs_BITUMEN_27132000,NaN,0,KBD,0.000000e+00
73,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,BITUMEN,cn_customs_BITUMEN_27132000,NaN,0,NC,1.053527e+09
2,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,CRUDEOIL,cn_customs_CRUDEOIL_27090000,NaN,0,KT,3.957939e+04


In [60]:
df_m2 = df_m[(df_m['unit'] == 'KBD') &
                    ~((df_m['Supplimentary Quantity'] == 0) & (df_m['Supplimentary Unit'].isna()))]\
            .drop(columns=['Supplimentary Quantity', 'Supplimentary Unit'])
df_m3 = df_m[~((df_m['unit'] == 'KBD') & (df_m['Supplimentary Quantity'] == 0) & (df_m['Supplimentary Unit'].isna()))]\
            .drop(columns=['Supplimentary Quantity', 'Supplimentary Unit'])

In [61]:
df_m3

,source,period,flow,product,detail,unit,value
0,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,MOTORGAS,cn_customs_MOTORGAS_27075000,KT,5.236950e+01
1,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,DIESEL,cn_customs_DIESEL_27079990,KT,2.921722e+02
2,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,CRUDEOIL,cn_customs_CRUDEOIL_27090000,KT,3.957939e+04
3,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,MOTORGAS,cn_customs_MOTORGAS_27101210,KT,3.771841e+01
4,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,NAPHTHA,cn_customs_NAPHTHA_27101220,KT,2.559505e+02
5,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,MOTORGAS,cn_customs_MOTORGAS_27101299,KT,1.616169e+01
6,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,JETKERO,cn_customs_JETKERO_27101911,KT,2.830425e+02
7,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,OTHKERO,cn_customs_OTHKERO_27101919,KT,4.731779e+00
8,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,RESFUEL,cn_customs_RESFUEL_27101922,KT,9.740762e+02
9,cn_customs_oilprod_imp_201906,JUN2019,IMPORTS,DIESEL,cn_customs_DIESEL_27101923,KT,1.365428e+02


In [29]:
df_rej = df[['Trading partner code', 'Trading partner']].drop_duplicates()
df_rej['Rejected Countries'] = df_rej['Trading partner code'].map(str) + " - " + df_rej["Trading partner"]
print(df_rej['Rejected Countries'].drop_duplicates().tolist())

['105 - Brunei', '111 - India', '112 - Indonesia', '113 - Iran', '114 - Iraq', '118 - Kuwait', '122 - Malaysia', '124 - Mongolia', '126 - Oman', '130 - Qatar', '131 - Saudi Arabia', '133 - Korea, Rep.', '136 - Thailand', '138 - United Arab Emirates', '139 - Yemen', '141 - Viet Nam', '145 - Kazakhstan', '201 - Algeria', '202 - Angola', '206 - Cameroon', '211 - Chad', '213 - Congo', '215 - Egypt', '216 - Equatorial Guinea', '218 - Gabon', '220 - Ghana', "223 - Cote d'lvoire", '226 - Libyan Arab Jamahiriya', '235 - Niger', '236 - Nigeria', '244 - South Africa', '246 - Sudan', '252 - Congo,DR', '260 - Republic of South Sudan ', '303 - United Kingdom', '326 - Norway', '339 - Azerbai jan', '344 - Russian Federation', '402 - Argentina', '410 - Brazil', '413 - Colombia', '419 - Ecuador', '429 - Mexico', '445 - Venezuela', '501 - Canada', '502 - United States', '601 - Australia', '143 - Taiwan, Prov.of China', '217 - Ethiopia', '313 - Albania']


In [78]:
cn.df.head()

AttributeError: 'CrudeImportsJob' object has no attribute 'df'

In [80]:
#cn.df = cn.df[['Date of data','code', 'Quantity', 'Renminbi Yuan']].rename(columns={'Date of data': 'period'})
cn.df.head()

,period,country,unit,value,provider,area,frequency,flow,original
0,201903,Iran,KT,2297.96,CN_CUSTOMS,CHINA,Monthly,IMPORTS,True
1,201903,Iraq,KT,3841.15,CN_CUSTOMS,CHINA,Monthly,IMPORTS,True
2,201903,Kuwait,KT,1804.13,CN_CUSTOMS,CHINA,Monthly,IMPORTS,True
3,201903,Malaysia,KT,508.132,CN_CUSTOMS,CHINA,Monthly,IMPORTS,True
4,201903,Mongolia,KT,73.6771,CN_CUSTOMS,CHINA,Monthly,IMPORTS,True


In [50]:
cn.df = cn.df.rename(columns={'Date of data': 'period', 'Quantity': 'KT', 'Renminbi Yuan': 'NC'})

In [53]:
cn.df['KT']  = cn.df['KT'] / 10 ** 6

In [192]:
df[df['country'].isnull()]

,area,country,flow,frequency,original,period,product,provider,source,unit,value


In [213]:
df = pd.DataFrame(cn.data)

In [197]:
import requests
r = requests.get('http://vipenta:8000/dimension/area')
area = pd.DataFrame(r.json())
df['country'].drop_duplicates().isin(area['code'])

0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
20     True
21     True
22     True
23     True
24     True
25     True
26     True
27     True
28     True
29     True
30     True
31     True
32     True
33     True
47     True
49     True
56     True
59     True
61     True
64     True
65     True
280    True
357    True
393    True
399    True
435    True
553    True
560    True
662    True
799    True
Name: country, dtype: bool

In [203]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1942 entries, 0 to 1941
Data columns (total 10 columns):
area         1942 non-null object
flow         1942 non-null object
frequency    1942 non-null object
original     1942 non-null bool
period       1942 non-null object
product      1942 non-null object
provider     1942 non-null object
source       1942 non-null object
unit         1942 non-null object
value        1942 non-null object
dtypes: bool(1), object(9)
memory usage: 138.5+ KB


In [204]:
df['period'] = df['period'].dt.strftime("%b%Y").str.upper()

AttributeError: Can only use .dt accessor with datetimelike values

In [214]:
df.query('provider')

,area,flow,frequency,original,period,product,provider,source,unit,value
0,CHINA,IMPORTS,Monthly,True,APR2019,CRUDEOIL,CN_CUSTOMS,cn_customs_crudeoil_imp_201904,KT,24.4648
1,CHINA,IMPORTS,Monthly,True,APR2019,CRUDEOIL,CN_CUSTOMS,cn_customs_crudeoil_imp_201904,KT,3243.03
2,CHINA,IMPORTS,Monthly,True,APR2019,CRUDEOIL,CN_CUSTOMS,cn_customs_crudeoil_imp_201904,KT,3313.86
3,CHINA,IMPORTS,Monthly,True,APR2019,CRUDEOIL,CN_CUSTOMS,cn_customs_crudeoil_imp_201904,KT,2382.98
4,CHINA,IMPORTS,Monthly,True,APR2019,CRUDEOIL,CN_CUSTOMS,cn_customs_crudeoil_imp_201904,KT,500.975
5,CHINA,IMPORTS,Monthly,True,APR2019,CRUDEOIL,CN_CUSTOMS,cn_customs_crudeoil_imp_201904,KT,75.348
6,CHINA,IMPORTS,Monthly,True,APR2019,CRUDEOIL,CN_CUSTOMS,cn_customs_crudeoil_imp_201904,KT,3066.41
7,CHINA,IMPORTS,Monthly,True,APR2019,CRUDEOIL,CN_CUSTOMS,cn_customs_crudeoil_imp_201904,KT,68.6841
8,CHINA,IMPORTS,Monthly,True,APR2019,CRUDEOIL,CN_CUSTOMS,cn_customs_crudeoil_imp_201904,KT,6296.69
9,CHINA,IMPORTS,Monthly,True,APR2019,CRUDEOIL,CN_CUSTOMS,cn_customs_crudeoil_imp_201904,KT,159.118


In [217]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1942 entries, 0 to 1941
Data columns (total 10 columns):
area         1942 non-null object
flow         1942 non-null object
frequency    1942 non-null object
original     1942 non-null bool
period       1942 non-null object
product      1942 non-null object
provider     1942 non-null object
source       1942 non-null object
unit         1942 non-null object
value        1942 non-null object
dtypes: bool(1), object(9)
memory usage: 138.5+ KB


In [223]:
df['value']

0             24.4648
1             3243.03
2             3313.86
3             2382.98
4             500.975
5              75.348
6             3066.41
7             68.6841
8             6296.69
9             159.118
10            778.168
11            138.641
12             143.32
13            280.009
14            4474.52
15            133.748
16            1134.14
17            67.2691
18            428.566
19            307.578
20            248.642
21            715.744
22            277.316
23            67.5274
24            144.625
25             1216.8
26             265.61
27            6123.31
28            3510.37
29            1447.07
            ...      
1912    4,486,162,833
1913    3,371,589,008
1914    2,030,075,270
1915      342,039,945
1916      165,361,097
1917      400,209,971
1918      360,726,401
1919              534
1920      433,170,339
1921    2,441,250,322
1922      804,181,649
1923    1,998,116,965
1924    2,098,677,653
1925      578,188,253
1926    2,

In [6]:
IMPORTS = 0 
EXPORTS = 1
FLOW = (IMPORTS, EXPORTS)
FLOW_CODE = ('IMPORTS', 'EXPORTS')

for flow in FLOW:
    print(FLOW_CODE[flow].lower()[:3])


imp
exp


In [2]:
import logging
logging.basicConfig(level = logging.DEBUG)

from scraper.core.job import BATCH_SIZE

import scraper.jobs.cn_customs.oil_products_job as job


logger = logging.getLogger(__name__)

logger.debug('test DEBUG')
logger.info('test INFO')

cn = job.OilProductsJob()
# cn.get_sources()
# cn.transform()
cn.run()

DEBUG:__main__:test DEBUG
INFO:__main__:test INFO
DEBUG:scraper.jobs.cn_customs.oil_products_job:Getting sources...
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): vipenta:8000


ConnectionError: HTTPConnectionPool(host='vipenta', port=8000): Max retries exceeded with url: /dimension/source (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000004E693CAEB8>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [7]:
cn.remove_existing_dynamic_dim('source')

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): vipenta:8000
DEBUG:urllib3.connectionpool:http://vipenta:8000 "GET /dimension/source HTTP/1.1" 200 570129


In [4]:
# cn.dynamic_dim['source']
cn.data

In [17]:
import pandas as pd
s = pd.Series(['lama', 'cow', 'lama', 'beetle', 'lama', 'hippo'], name='animal')
if not s.isin(['lama']).all():
    print('KO')

KO


In [124]:
from scraper.settings import FILE_STORE_PATH

file_path = FILE_STORE_PATH / 'cn_customs_oilprod_imp_2018.csv'
df = pd.read_csv(file_path, thousands=job.THOUSANDS, encoding='windows-1252', na_values='-')

df[~((df['Supplimentary Quantity'] ==  0) & (df['Supplimentary Unit'].isna()))]
#df[~(df['Supplimentary Unit'].isna() | (df['Supplimentary Unit'] == 'Litre'))]['Supplimentary Unit'].drop_duplicates().tolist()
#df['Supplimentary Unit'].drop_duplicates()

,Date of data,Commodity code,Commodity,Quantity,Unit,Supplimentary Quantity,Supplimentary Unit,Renminbi Yuan,Unnamed: 8
36,201802,27101210,"Motor/aviation gasoline, without biodiesel",39908845,Kilogram,55393477,Litre,176903199,NaN
37,201805,27101210,"Motor/aviation gasoline, without biodiesel",19591,Kilogram,27192,Litre,283049,NaN
38,201808,27101210,"Motor/aviation gasoline, without biodiesel",81375257,Kilogram,112948857,Litre,373589431,NaN
39,201811,27101210,"Motor/aviation gasoline, without biodiesel",70635453,Kilogram,98042009,Litre,335897881,NaN
40,201801,27101210,"Motor/aviation gasoline, without biodiesel",14960,Kilogram,20764,Litre,277670,NaN
41,201804,27101210,"Motor/aviation gasoline, without biodiesel",28540,Kilogram,39613,Litre,528914,NaN
42,201807,27101210,"Motor/aviation gasoline, without biodiesel",865,Kilogram,1200,Litre,42521,NaN
43,201810,27101210,"Motor/aviation gasoline, without biodiesel",164008436,Kilogram,227643710,Litre,856346237,NaN
44,201803,27101210,"Motor/aviation gasoline, without biodiesel",25623,Kilogram,35565,Litre,707221,NaN
45,201806,27101210,"Motor/aviation gasoline, without biodiesel",14960,Kilogram,20764,Litre,281529,NaN


In [73]:
1 / 158.987294928

0.006289810770432105

In [103]:
#df['Date of data'].apply(lambda x: pd.to_datetime(x, format='%Y%m')).dt.strftime("%m").astype('int')
df['year'] = df['Date of data'].apply(lambda x: x // 100)
df['month'] = df['Date of data'].apply(lambda x: x % 100)


df

,Date of data,Commodity code,Commodity,Quantity,Unit,Supplimentary Quantity,Supplimentary Unit,Renminbi Yuan,Unnamed: 8,period,year,month
0,201809,27075000,Other aromatic hydrocarbon mixtures,195985811,Kilogram,0,NaN,947308588,NaN,SEP2018,2018,9
1,201812,27075000,Other aromatic hydrocarbon mixtures,302949948,Kilogram,0,NaN,1343930051,NaN,DEC2018,2018,12
2,201803,27075000,Other aromatic hydrocarbon mixtures,451624520,Kilogram,0,NaN,2077863827,NaN,MAR2018,2018,3
3,201806,27075000,Other aromatic hydrocarbon mixtures,529399798,Kilogram,0,NaN,2570538890,NaN,JUN2018,2018,6
4,201801,27075000,Other aromatic hydrocarbon mixtures,736583357,Kilogram,0,NaN,3485185619,NaN,JAN2018,2018,1
5,201804,27075000,Other aromatic hydrocarbon mixtures,325707327,Kilogram,0,NaN,1549672091,NaN,APR2018,2018,4
6,201807,27075000,Other aromatic hydrocarbon mixtures,425242763,Kilogram,0,NaN,2119398713,NaN,JUL2018,2018,7
7,201810,27075000,Other aromatic hydrocarbon mixtures,193526196,Kilogram,0,NaN,1023540472,NaN,OCT2018,2018,10
8,201802,27075000,Other aromatic hydrocarbon mixtures,795168855,Kilogram,0,NaN,3641202592,NaN,FEB2018,2018,2
9,201805,27075000,Other aromatic hydrocarbon mixtures,382013919,Kilogram,0,NaN,1751473168,NaN,MAY2018,2018,5


In [ ]:
from scraper.jobs.utils import convert_m3_to_kbd

p